<a href="https://colab.research.google.com/github/Curiousss/EVAAssignment7/blob/master/TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import pprint # for pretty printing our device stats

if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
      devices = session.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

TPU address is grpc://10.94.244.122:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 10719749730960052620),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9751140309556774338),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 3630012588680747504),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 10199871570562237856),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 503829590128109815),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 15127912793016231470),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 11724721852201664558),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 9052286802375921412),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 211398431931

In [2]:
resolver = tf.contrib.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

W0729 12:12:06.497432 140476372449152 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# preprocessing the dataset
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)

(60000, 28, 28) (60000, 10)
(10000, 28, 28) (10000, 10)


In [0]:
x_train = x_train.reshape(x_train.shape[0], 28, 28,1)
x_test = x_test.reshape(x_test.shape[0], 28, 28,1)

In [5]:
#from tf.keras.layers import Convolution2D, MaxPooling2D, Activation, BatchNormalization, Flatten
with strategy.scope():

  dropout=0.15

  model = tf.keras.models.Sequential()


  model.add(tf.keras.layers.Convolution2D(10, 3, activation='relu', input_shape=(28,28,1))) #26
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #24
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #22
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  # Bottleneck layer
  model.add(tf.keras.layers.MaxPooling2D()) #11
  model.add(tf.keras.layers.Convolution2D(10, 1, activation='relu')) #11
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #9
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(16, 3, activation='relu')) #7
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  # 1x1 Conv
  model.add(tf.keras.layers.Convolution2D(10, 1, activation='relu')) #7
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dropout(dropout))

  model.add(tf.keras.layers.Convolution2D(10, 7))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Activation('softmax'))
  model.summary()
  model.compile(
        optimizer=tf.train.AdamOptimizer(learning_rate=1e-2),
        loss=tf.keras.losses.categorical_crossentropy,
        metrics=['accuracy'])


W0729 12:12:19.660348 140476372449152 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 10)        40        
_________________________________________________________________
dropout (Dropout)            (None, 26, 26, 10)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 16)        1456      
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 16)        64        
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 16)        2

In [8]:
history = model.fit(x_train, 
		                y_train,
                    epochs=10,
                    validation_data=(x_test, y_test),
                    steps_per_epoch=50,
#                    validation_steps=1,
                    verbose=1
                    )

model.save_weights('./mnist_model.h5', overwrite=True)

Epoch 1/10
50/50 [==============================] - 49s 985ms/step - loss: 0.0251 - acc: 0.9920 - val_loss: 0.0207 - val_acc: 0.9931
Epoch 2/10
50/50 [==============================] - 45s 896ms/step - loss: 0.0230 - acc: 0.9925 - val_loss: 0.0225 - val_acc: 0.9919
Epoch 3/10
50/50 [==============================] - 46s 923ms/step - loss: 0.0250 - acc: 0.9915 - val_loss: 0.0190 - val_acc: 0.9932
Epoch 4/10
50/50 [==============================] - 47s 943ms/step - loss: 0.0234 - acc: 0.9922 - val_loss: 0.0205 - val_acc: 0.9929
Epoch 5/10
50/50 [==============================] - 49s 984ms/step - loss: 0.0248 - acc: 0.9919 - val_loss: 0.0222 - val_acc: 0.9921
Epoch 6/10
50/50 [==============================] - 49s 985ms/step - loss: 0.0253 - acc: 0.9918 - val_loss: 0.0218 - val_acc: 0.9929
Epoch 7/10
50/50 [==============================] - 56s 1s/step - loss: 0.0238 - acc: 0.9922 - val_loss: 0.0194 - val_acc: 0.9937
Epoch 8/10
50/50 [==============================] - 59s 1s/step - loss: 